Ethics of AI Final Project

In [18]:
!pip install datasets

In [19]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import  DecisionTreeClassifier ,plot_tree, export_text# Decision tree algorithm and plotting functions for the Decision tree
from matplotlib import pyplot as plt # plotting/graphing
import numpy as np
from itertools import product
from sklearn.utils import class_weight


In [31]:
# fetch dataset and apply processing file
dataset = load_dataset("mstz/speeddating",trust_remote_code=True)["train"]

# convert to df
df = pd.DataFrame(dataset)


# split into x and y sets
X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

X.columns


Index(['is_dater_male', 'dater_age', 'dated_age', 'age_difference',
       'are_same_race', 'same_race_importance_for_dater',
       'same_religion_importance_for_dater',
       'attractiveness_importance_for_dated', 'sincerity_importance_for_dated',
       'intelligence_importance_for_dated', 'humor_importance_for_dated',
       'ambition_importance_for_dated',
       'shared_interests_importance_for_dated',
       'attractiveness_score_of_dater_from_dated',
       'sincerity_score_of_dater_from_dated',
       'intelligence_score_of_dater_from_dated',
       'humor_score_of_dater_from_dated', 'ambition_score_of_dater_from_dated',
       'shared_interests_score_of_dater_from_dated',
       'attractiveness_importance_for_dater', 'sincerity_importance_for_dater',
       'intelligence_importance_for_dater', 'humor_importance_for_dater',
       'ambition_importance_for_dater',
       'shared_interests_importance_for_dater',
       'self_reported_attractiveness_of_dater',
       'self_repor

In [32]:
# split into training and tests, ALL RANDOM STATES SET TO 0
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .33,random_state=0)

In [33]:
positive_weight = 8  # You can adjust this weight as needed
negative_weight = 1  # You can adjust this weight as needed

# Create a dictionary to specify class weights
class_weights = {0: negative_weight, 1: positive_weight}

In [34]:
# Different models
# Random Forest

#Set estimators as 100, random state 0, samples of at least 2 in each leaf
clf = RandomForestClassifier(class_weight={0: class_weights[0], 1: class_weights[1]},n_estimators = 200, random_state=0,min_samples_leaf=6)
clf.fit(x_train, y_train) # fit model

# get the roc auc and pring
cross_val_accuracy_roc_auc = (cross_val_score(clf, x_train, y_train, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_accuracy_roc_auc

84.49718460607754

In [35]:
results = x_test.copy()
y_hat = clf.predict(results)
results['predicted'] = y_hat
results['actual'] = y_test

In [36]:
dated_groups = ["dated_race_'Asian/Pacific Islander/Asian-American'","dated_race_'Black/African American'","dated_race_'Latino/Hispanic American'", 'dated_race_caucasian','dated_race_other']

def safe_divide(n, d):
  try:
    return n/d
  except ZeroDivisionError:
    return 0

def compute_metrics(info, group):
  metric_df = info.loc[info[group] == 1]
  pred_pos = sum(metric_df['predicted'])
  act_pos = sum(metric_df['actual'])

  positive_rate = pred_pos/len(metric_df)
  accuracy = len(metric_df[metric_df['predicted'] == metric_df['actual']])/len(metric_df)
  tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
  fp = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 1)])
  fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
  tn = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 0)])
  tpr = safe_divide(tp,(tp + fn ))
  fpr = safe_divide(fp,(fp + tn))
  precision = safe_divide(tp, (tp + fp))
  recall = safe_divide(tp,(tp + fn))

  row = [group, accuracy, precision, recall, positive_rate, tpr, fpr]

  return row

column_names = ['group', 'accuracy', 'precision', 'recall', 'positive_rate', 'tpr', 'fpr']
metrics_df = pd.DataFrame(columns=column_names)

for group in dated_groups:
  row = compute_metrics(results, group)
  metrics_df.loc[len(metrics_df)] = row

metrics_df

,group,accuracy,precision,recall,positive_rate,tpr,fpr
0,dated_race_'Asian/Pacific Islander/Asian-Ameri...,0.840000,0.181818,0.400000,0.146667,0.400000,0.128571
1,dated_race_'Black/African American',0.875000,0.800000,0.666667,0.208333,0.666667,0.055556
2,dated_race_'Latino/Hispanic American',0.888889,0.750000,0.600000,0.148148,0.600000,0.045455
3,dated_race_caucasian,0.836735,0.513514,0.575758,0.188776,0.575758,0.110429
4,dated_race_other,0.791667,0.625000,0.714286,0.333333,0.714286,0.176471


# 1 Pre Processing
Our dataset has a variety of features. We will want to simplify our dataset to have a few key features:
Age, age difference for the pairing (Sensitive)
Race, same race for the pairing (Sensitive)
expected_number_of_likes_of_dater_from_20_people (non sensitive)
Already met before (non sensitive)


In [43]:
# convert to df
df = pd.DataFrame(dataset)
df = df.groupby('dated_race').apply(lambda x: x.sample(n=196, replace=True)).reset_index(drop=True) # Replace n with size of group of interest


# split into x and y sets
X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

In [44]:
# split into training and tests, ALL RANDOM STATES SET TO 0
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .33,random_state=0)

In [45]:
positive_weight = 8  # You can adjust this weight as needed
negative_weight = 1  # You can adjust this weight as needed

# Create a dictionary to specify class weights
class_weights = {0: negative_weight, 1: positive_weight}

In [46]:
clf = RandomForestClassifier(class_weight={0: class_weights[0], 1: class_weights[1]},n_estimators = 200, random_state=0,min_samples_leaf=6)
clf.fit(x_train, y_train) # fit model

# get the roc auc and pring
cross_val_accuracy_roc_auc = (cross_val_score(clf, x_train, y_train, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_accuracy_roc_auc

94.6011499385955

In [47]:
results = x_test.copy()
y_hat = clf.predict(results)
results['predicted'] = y_hat
results['actual'] = y_test

In [48]:
dated_groups = ["dated_race_'Asian/Pacific Islander/Asian-American'","dated_race_'Black/African American'","dated_race_'Latino/Hispanic American'", 'dated_race_caucasian','dated_race_other']

def safe_divide(n, d):
  try:
    return n/d
  except ZeroDivisionError:
    return 0

def compute_metrics(info, group):
  metric_df = info.loc[info[group] == 1]
  pred_pos = sum(metric_df['predicted'])
  act_pos = sum(metric_df['actual'])

  positive_rate = pred_pos/len(metric_df)
  accuracy = len(metric_df[metric_df['predicted'] == metric_df['actual']])/len(metric_df)
  tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
  fp = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 1)])
  fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
  tn = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 0)])
  tpr = safe_divide(tp,(tp + fn ))
  fpr = safe_divide(fp,(fp + tn))
  precision = safe_divide(tp, (tp + fp))
  recall = safe_divide(tp,(tp + fn))

  row = [group, accuracy, precision, recall, positive_rate, tpr, fpr]

  return row

column_names = ['group', 'accuracy', 'precision', 'recall', 'positive_rate', 'tpr', 'fpr']
metrics_df = pd.DataFrame(columns=column_names)

for group in dated_groups:
  row = compute_metrics(results, group)
  metrics_df.loc[len(metrics_df)] = row

metrics_df

,group,accuracy,precision,recall,positive_rate,tpr,fpr
0,dated_race_'Asian/Pacific Islander/Asian-Ameri...,0.982759,0.750000,1.000000,0.068966,1.000000,0.018182
1,dated_race_'Black/African American',0.971429,0.833333,1.000000,0.171429,1.000000,0.033333
2,dated_race_'Latino/Hispanic American',0.904762,0.600000,1.000000,0.238095,1.000000,0.111111
3,dated_race_caucasian,0.875000,0.461538,0.857143,0.203125,0.857143,0.122807
4,dated_race_other,0.956522,0.909091,0.833333,0.159420,0.833333,0.017544


## 2 - Inprocess Bias Mitigation

In [51]:
df = pd.DataFrame(dataset)

X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

# Same race data split into y and X
X_same = df[df['are_same_race'] == 1]
y_same = X_same.is_match
X_same = X_same.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date'])
X_same = pd.get_dummies(X_same) #get dummies for X

# Different race data split into y and x
X_diff = df[df['are_same_race'] == 0]
y_diff = X_diff.is_match
X_diff = X_diff.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date'])
X_diff = pd.get_dummies(X_diff) #get dummies for X

x_train_same, x_test_same, y_train_same, y_test_same = train_test_split(X_same, y_same, test_size = .33,random_state=0)
x_train_diff, x_test_diff, y_train_diff, y_test_diff = train_test_split(X_diff, y_diff, test_size = .33,random_state=0)

clf1 = RandomForestClassifier(class_weight={0: class_weights[0], 1: class_weights[1]},n_estimators = 200, random_state=0,min_samples_leaf=6)
clf1.fit(x_train_same, y_train_same) # fit model

clf2 = RandomForestClassifier(class_weight={0: class_weights[0], 1: class_weights[1]},n_estimators = 200, random_state=0,min_samples_leaf=6)
clf2.fit(x_train_diff, y_train_diff) # fit model

cross_val_dict = {}
# get the roc auc and pring
# Diff
cross_val_accuracy_roc_auc = (cross_val_score(clf1, x_train_same, y_train_same, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_dict['same'] = cross_val_accuracy_roc_auc

# Diff
cross_val_accuracy_roc_auc = (cross_val_score(clf1, x_train_diff, y_train_diff, cv = 10, scoring = 'roc_auc').mean()*100)

cross_val_dict['diff'] = cross_val_accuracy_roc_auc

print(cross_val_dict)


{'same': 85.76147342995168, 'diff': 85.38706391647567}


In [52]:
# Same results
results_same = x_test_same.copy()
y_hat = clf1.predict(results_same)
results_same['predicted'] = y_hat
results_same['actual'] = y_test_same

# Diff results
results_diff = x_test_diff.copy()
y_hat = clf2.predict(results_diff)
results_diff['predicted'] = y_hat
results_diff['actual'] = y_test_diff

results = pd.concat([results_same, results_diff])

In [ ]:
results.head()

,is_dater_male,dater_age,dated_age,age_difference,are_same_race,same_race_importance_for_dater,same_religion_importance_for_dater,attractiveness_importance_for_dated,sincerity_importance_for_dated,intelligence_importance_for_dated,...,dater_race_'Latino/Hispanic American',dater_race_caucasian,dater_race_other,dated_race_'Asian/Pacific Islander/Asian-American',dated_race_'Black/African American',dated_race_'Latino/Hispanic American',dated_race_caucasian,dated_race_other,predicted,actual
601,False,34,24,10,True,5.0,7.0,10.0,20.0,35.0,...,0,1,0,0,0,0,1,0,0,0
694,False,30,26,4,True,3.0,5.0,18.0,18.0,18.0,...,0,1,0,0,0,0,1,0,0,0
764,False,29,26,3,True,1.0,2.0,18.0,18.0,18.0,...,0,1,0,0,0,0,1,0,0,0
453,True,28,23,5,True,2.0,2.0,10.0,20.0,20.0,...,0,1,0,0,0,0,1,0,0,0
149,False,27,30,3,True,2.0,3.0,20.0,20.0,25.0,...,0,1,0,0,0,0,1,0,0,0


In [53]:
dated_groups = ["dated_race_'Asian/Pacific Islander/Asian-American'","dated_race_'Black/African American'","dated_race_'Latino/Hispanic American'", 'dated_race_caucasian','dated_race_other']

def safe_divide(n, d):
  try:
    return n/d
  except ZeroDivisionError:
    return np.nan


def compute_metrics(info, group):
  metric_df = info.loc[info[group] == 1]
  pred_pos = sum(metric_df['predicted'])
  act_pos = sum(metric_df['actual'])

  positive_rate = pred_pos/len(metric_df)
  accuracy = len(metric_df[metric_df['predicted'] == metric_df['actual']])/len(metric_df)
  tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
  fp = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 1)])
  fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
  tn = len(metric_df[(metric_df['actual'] == 0) & (metric_df['predicted'] == 0)])
  tpr = safe_divide(tp, (tp + fn))
  fpr = safe_divide(fp, (fp + tn))
  precision = safe_divide(tp, (tp + fp))
  recall = safe_divide(tp, (tp + fn))

  row = [group, accuracy, precision, recall, positive_rate, tpr, fpr]

  return row

column_names = ['group', 'accuracy', 'precision', 'recall', 'positive_rate', 'tpr', 'fpr']
metrics_df = pd.DataFrame(columns=column_names)

for group in dated_groups:
  row = compute_metrics(results, group)
  metrics_df.loc[len(metrics_df)] = row

metrics_df






,group,accuracy,precision,recall,positive_rate,tpr,fpr
0,dated_race_'Asian/Pacific Islander/Asian-Ameri...,0.863014,0.444444,0.444444,0.123288,0.444444,0.078125
1,dated_race_'Black/African American',0.888889,0.750000,1.000000,0.444444,1.000000,0.166667
2,dated_race_'Latino/Hispanic American',0.826087,0.500000,0.500000,0.173913,0.500000,0.105263
3,dated_race_caucasian,0.801843,0.410256,0.444444,0.179724,0.444444,0.127072
4,dated_race_other,0.937500,1.000000,0.800000,0.250000,0.800000,0.000000


## 3 - Post-Processing

In [ ]:

# split into x and y sets
X = df.drop(columns=['is_match','dater_wants_to_date','dated_wants_to_date']) # is match is our target variable, which is determined from the last 2 so we need to drop all
X = pd.get_dummies(X) #get dummies for X
y = df.is_match

# split into training and tests, ALL RANDOM STATES SET TO 0
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .33,random_state=0)
#Set estimators as 100, random state 0, samples of at least 2 in each leaf
clf = RandomForestClassifier(class_weight={0: class_weights[0], 1: class_weights[1]},n_estimators = 200, random_state=0,min_samples_leaf=6)
clf.fit(x_train, y_train) # fit model


RandomForestClassifier(class_weight={0: 1, 1: 8}, min_samples_leaf=6,
                       n_estimators=200, random_state=0)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, roc_auc_score # Various model evaluation metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict # train test split and cross validation accuracy/prediction functions

results = X
y_hat = clf.predict(X)
y_hat_proba = clf.predict_proba(X)

results['predicted'] = y_hat
# Append predicted probabilities to the results DataFrame
results['predicted_prob'] = y_hat_proba[:, 1]
results['actual'] = y_test


predictions = cross_val_predict(clf, X, y, cv = 10)
confusion = confusion_matrix(y, predictions)
tn, fp, fn, tp = confusion.ravel()

In [ ]:
#we see the rates for each metric of the entire df

print('True Negatives:', tn)
print('False positives:', fp)
print('False Negatives',fn)
print('True Positives',tp)

True Negatives: 811
False positives: 51
False Negatives 28
True Positives 158


In [ ]:
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'","dater_race_'Black/African American'","dater_race_'Latino/Hispanic American'", 'dater_race_caucasian','dater_race_other']

def safe_divide(n, d):
  try:
    return n/d
  except ZeroDivisionError:
    return np.nan


def compute_metrics(info, group):
    metric_df = info.loc[info[group] == 1]
    tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
    fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
    recall = safe_divide(tp, (tp + fn))
    return recall

column_names = ['group', 'recall']
metrics_df2 = pd.DataFrame(columns=column_names)
for group in dater_groups:
    recall = compute_metrics(results, group)
    metrics_df2.loc[len(metrics_df2)] = [group, recall]

metrics_df2



,group,recall
0,dater_race_'Asian/Pacific Islander/Asian-Ameri...,0.692308
1,dater_race_'Black/African American',1.000000
2,dater_race_'Latino/Hispanic American',0.333333
3,dater_race_caucasian,0.548387
4,dater_race_other,0.750000


In [ ]:
import numpy as np
import pandas as pd

# Function to compute recall
def safe_divide(n, d):
    try:
        return n / d
    except ZeroDivisionError:
        return np.nan

# Function to compute metrics
def compute_metrics(info, group):
    metric_df = info.loc[info[group] == 1]
    print(metric_df)
    tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 1)])
    fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted'] == 0)])
    recall = safe_divide(tp, (tp + fn))
    return recall

# Function to compute recall using custom threshold
def compute_recall_with_threshold(info, group, threshold):
    metric_df = info.loc[info[group] == 1]

    tp = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted_prob'] > threshold)])
    fn = len(metric_df[(metric_df['actual'] == 1) & (metric_df['predicted_prob'] <= threshold)])
    recall = safe_divide(tp, (tp + fn))
    return recall

# Define the list of race groups
dater_groups = ["dater_race_'Asian/Pacific Islander/Asian-American'",
                "dater_race_'Black/African American'",
                "dater_race_'Latino/Hispanic American'",
                'dater_race_caucasian',
                'dater_race_other']

# Initialize an empty DataFrame to store results
metrics_df = pd.DataFrame()

# Assuming you have your list of recall thresholds named 'recall_thresholds'
recall_thresholds = [0.4, .5, 0.25, 0.3, 0.5]

# Iterate over each race group
for group, threshold in zip(dater_groups, recall_thresholds):
    # Compute recall for the group with the custom threshold
    og_recall = compute_recall_with_threshold(results, group, .5)
    recall = compute_recall_with_threshold(results, group, threshold)
    # Add the computed recall and the corresponding threshold to the DataFrame
    metrics_df = metrics_df.append({'group': group,  'original_recall': og_recall, 'recall_threshold': threshold, 'new_recall': recall}, ignore_index=True)

# Print the DataFrame
(metrics_df)


C:\Users\arian\AppData\Local\Temp\ipykernel_10824\300813177.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({'group': group,  'original_recall': og_recall, 'recall_threshold': threshold, 'new_recall': recall}, ignore_index=True)
C:\Users\arian\AppData\Local\Temp\ipykernel_10824\300813177.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({'group': group,  'original_recall': og_recall, 'recall_threshold': threshold, 'new_recall': recall}, ignore_index=True)
C:\Users\arian\AppData\Local\Temp\ipykernel_10824\300813177.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({'group': group,  'original_recall': og_recall, 'recall_thr

,group,original_recall,recall_threshold,new_recall
0,dater_race_'Asian/Pacific Islander/Asian-Ameri...,0.692308,0.40,0.846154
1,dater_race_'Black/African American',1.000000,0.50,1.000000
2,dater_race_'Latino/Hispanic American',0.333333,0.25,0.833333
3,dater_race_caucasian,0.548387,0.30,0.838710
4,dater_race_other,0.750000,0.50,0.750000
